In [51]:
import re
import pandas as pd
import numpy as np
import subprocess
import scipy.stats as st
from scipy.optimize import minimize
import scipy.stats as stats
from scipy.interpolate import make_interp_spline
import seaborn
from os import listdir
from os.path import isfile, join
from functools import reduce
import itertools

from liftover import get_lifter

In [3]:
# Pre-221119
cd /Users/davidy/jamboree20crispr/ontarget_analysis/220505_epifeature_intersection

/Users/davidy/jamboree20crispr/ontarget_analysis/220505_epifeature_intersection


In [4]:
cd /Users/davidy/jamboree20crispr/ontarget_analysis/221119_epifeature_intersection

/Users/davidy/jamboree20crispr/ontarget_analysis/221119_epifeature_intersection


In [5]:
### I/O

all_files = [f for f in listdir('/Users/davidy/jamboree20crispr/ontarget_analysis/all_series') if isfile(join('/Users/davidy/jamboree20crispr/ontarget_analysis/all_series', f))]
eq_files = [f for f in all_files if '_EQ_' in f]

gene_list = set([f.split('_')[0] for f in eq_files])

hg38ensembl_promoter_dict = {} # Key = Gene; Value = promoter chromcoord


In [35]:
##### Alternative promoter dataframe
hg38_mart = pd.read_csv('/Users/davidy/misc_resources/hg38_mart_export.txt', sep = ' ', skiprows=1, header = None, names = ['ensg','symbol','type','chr','gene_start','gene_end','tss','tss_start','tss_end'])
lod = []
for i, row in hg38_mart.iterrows():
#if row.type == 'protein_coding':
    if row.tss == row.tss_start:
        strand = '+'
        start = row.tss_start-1000
        end = row.tss_start+1000
    else:
        strand = '-'
        start = row.tss_end-1000
        end = row.tss_end+1000

        #adict = {'chr':row.chr,
                #'start':start,
                #'end':end,
                #'gene':row.symbol}

        #lod.append(adict)

    # Record cc 
    chromcoord = row.chr + '_' + str(start) + '_' + str(end)
    hg38ensembl_promoter_dict.setdefault(row.symbol, []).append(chromcoord)

    #else:
    #    continue

In [20]:
non_epifeature_overlapping_sig_elements = set()
all_sig_elements = set()
nonpromoter_epifeature_overlapping_sig_elements = set()
all_element_df = pd.DataFrame()
full_all_element_df = pd.DataFrame()

##### Begin iterating over genes
all_gene_df = pd.DataFrame()

for gene in gene_list:
    print(gene)
    ### Process the element_quantification file
    eq_file = '/Users/davidy/jamboree20crispr/ontarget_analysis/all_series' + '/' + [f for f in eq_files if re.match(r'{}'.format(gene), f)][0]
    eq_elements = pd.read_csv(eq_file, sep = '\t', usecols=[0,1,2,3], names = ['chr','start','end','element'])
    print(eq_elements)
    
    ## Track all significant elements.
    # For intersecting elements (for Lexi)
    # For determining frequency of +/- 100 kb
    # For determining frequency of ABC
    temp_eq_elements = eq_elements
    temp_eq_elements['gene'] = gene
    all_element_df = pd.concat([all_element_df, eq_elements])
    full_all_element_df = pd.concat([full_all_element_df, temp_eq_elements])
    
    # If there are no significant elements, skip the gene
    if eq_elements.empty:
        continue
        
    for element in eq_elements['element'].tolist():
        all_sig_elements.add(element)

    # Get promoter peaks
    promoter_peaks = []

    if gene in hg38ensembl_promoter_dict:
        for cc in hg38ensembl_promoter_dict[gene]:
            pro_start, pro_end = cc.split('_')[1:]
            for row in eq_elements.itertuples():
                peak_start, peak_end = row.start, row.end
                if set(range(int(peak_start), int(peak_end))) & set(range(int(pro_start), int(pro_end))):
                    promoter_peaks.append(row.element)
    else:
        pass

    ### Intersect with peaks
    # DHS intersection
    eq_intersect_file = 'eq_intersect.bed'
    fopen = open(eq_intersect_file, 'w')
    subprocess.call(['bedtools','intersect', '-a', eq_file, '-b', '/Users/davidy/misc_resources/chip/results/macs2/dhs_k562_hg38_optimal_peak_unique.bed', '-wa', '-wb'], stdout = fopen)
    fopen.close()
    eq_dhs_intersect_df = pd.read_csv(eq_intersect_file, sep = '\t', usecols=[0,1,2,3, 25,26,27], names = ['chr','start','end','element','chr2','start2','end2'])
    eq_dhs_intersect_df['epifeature'] = 'DHS'

    # ATAC intersection
    eq_intersect_file = 'eq_intersect.bed'
    fopen = open(eq_intersect_file, 'w')
    subprocess.call(['bedtools','intersect', '-a', eq_file, '-b', '/Users/davidy/misc_resources/chip/results/macs2/atac_k562_hg38_unique.bed', '-wa', '-wb'], stdout = fopen)
    fopen.close()
    eq_atac_intersect_df = pd.read_csv(eq_intersect_file, sep = '\t', usecols=[0,1,2,3, 25,26,27], names = ['chr','start','end','element','chr2','start2','end2'])
    eq_atac_intersect_df['epifeature'] = 'ATAC'

    # H3K27ac intersection
    eq_intersect_file = 'eq_intersect.bed'
    fopen = open(eq_intersect_file, 'w')
    subprocess.call(['bedtools','intersect', '-a', eq_file, '-b', '/Users/davidy/misc_resources/chip/results/macs2/h3k27ac_k562_hg38_unique.bed', '-wa', '-wb'], stdout = fopen)
    fopen.close()
    eq_h3k27ac_intersect_df = pd.read_csv(eq_intersect_file, sep = '\t', usecols=[0,1,2,3, 25,26,27], names = ['chr','start','end','element','chr2','start2','end2'])
    eq_h3k27ac_intersect_df['epifeature'] = 'H3K27ac'

    # Concatenate all 3 dataframes
    eq_allepifeature_intersect_df = pd.concat([eq_dhs_intersect_df, eq_atac_intersect_df, eq_h3k27ac_intersect_df])
    eq_allepifeature_intersect_df['gene'] = gene

    # Annotate with whether it's promoter overlapping or not
    eq_allepifeature_intersect_df['overlaps_tss'] = np.where(eq_allepifeature_intersect_df['element'].isin(promoter_peaks), 'promoter', 'enhancer')

    ### Determine if any elements do not overlap DHS/H3K27ac
    non_epifeature = list(set(eq_elements['element'].tolist()).difference(eq_allepifeature_intersect_df['element'].tolist()))
    print(non_epifeature)
    display(eq_allepifeature_intersect_df)
    for element2 in non_epifeature:
        non_epifeature_overlapping_sig_elements.add(element2)
    
    
    # Add the dataframe to the master dataframe
    all_gene_df = pd.concat([all_gene_df, eq_allepifeature_intersect_df])

# Write out all gene dataframe
all_gene_df.to_csv('AllGene_Epifeature_Intersection.bed', sep = '\t', index = False, header = False)

print('Number of all sig elements {}'.format(len(all_sig_elements)))
print('Number of sig elements that dont overlap epifeature {}'.format(len(non_epifeature_overlapping_sig_elements)))

display(all_element_df)
all_element_df.to_csv('All_Sabeti_HCRFF_Elements.bed', sep = '\t', index=False, header=False)
display(full_all_element_df)


CAT
     chr     start       end                    element
0  chr11  33944868  33945268  chr11:33944868-33945268:.
1  chr11  34437468  34440568  chr11:34437468-34440568:.
[]


,chr,start,end,element,chr2,start2,end2,epifeature,gene,overlaps_tss
0,chr11,33944868,33945268,chr11:33944868-33945268:.,chr11,33944500,33945221,DHS,CAT,enhancer
1,chr11,34437468,34440568,chr11:34437468-34440568:.,chr11,34438478,34439511,DHS,CAT,promoter
0,chr11,33944868,33945268,chr11:33944868-33945268:.,chr11,33944535,33945267,ATAC,CAT,enhancer
1,chr11,34437468,34440568,chr11:34437468-34440568:.,chr11,34438477,34439525,ATAC,CAT,promoter
0,chr11,33944868,33945268,chr11:33944868-33945268:.,chr11,33939962,33946587,H3K27ac,CAT,enhancer
1,chr11,34437468,34440568,chr11:34437468-34440568:.,chr11,34437324,34437799,H3K27ac,CAT,promoter
2,chr11,34437468,34440568,chr11:34437468-34440568:.,chr11,34437880,34438630,H3K27ac,CAT,promoter
3,chr11,34437468,34440568,chr11:34437468-34440568:.,chr11,34438848,34440485,H3K27ac,CAT,promoter


LMO2
     chr     start       end                    element
0  chr11  33869368  33870768  chr11:33869368-33870768:.
1  chr11  33880468  33883368  chr11:33880468-33883368:.
2  chr11  33937168  33938068  chr11:33937168-33938068:.
3  chr11  33940268  33943268  chr11:33940268-33943268:.
4  chr11  33943668  33946568  chr11:33943668-33946568:.
5  chr11  33947368  33947868  chr11:33947368-33947868:.
[]


,chr,start,end,element,chr2,start2,end2,epifeature,gene,overlaps_tss
0,chr11,33869368,33870768,chr11:33869368-33870768:.,chr11,33869527,33869751,DHS,LMO2,promoter
1,chr11,33869368,33870768,chr11:33869368-33870768:.,chr11,33869786,33870019,DHS,LMO2,promoter
2,chr11,33869368,33870768,chr11:33869368-33870768:.,chr11,33870097,33871195,DHS,LMO2,promoter
3,chr11,33880468,33883368,chr11:33880468-33883368:.,chr11,33881952,33882395,DHS,LMO2,enhancer
4,chr11,33937168,33938068,chr11:33937168-33938068:.,chr11,33936857,33937690,DHS,LMO2,enhancer
5,chr11,33940268,33943268,chr11:33940268-33943268:.,chr11,33941604,33942092,DHS,LMO2,enhancer
6,chr11,33940268,33943268,chr11:33940268-33943268:.,chr11,33942939,33943233,DHS,LMO2,enhancer
7,chr11,33943668,33946568,chr11:33943668-33946568:.,chr11,33944500,33945221,DHS,LMO2,enhancer
0,chr11,33869368,33870768,chr11:33869368-33870768:.,chr11,33868514,33871508,ATAC,LMO2,promoter
1,chr11,33880468,33883368,chr11:33880468-33883368:.,chr11,33882029,33882374,ATAC,LMO2,enhancer


HDAC6
    chr     start       end                   element
0  chrX  48786297  48786997  chrX:48786297-48786997:.
1  chrX  48800297  48801097  chrX:48800297-48801097:.
2  chrX  48801697  48802997  chrX:48801697-48802997:.
[]


...

MYC
    chr      start        end                     element
0  chr8  127735369  127736269  chr8:127735369-127736269:.
1  chr8  127794569  127795269  chr8:127794569-127795269:.
2  chr8  127898369  127899769  chr8:127898369-127899769:.
3  chr8  127959469  127961769  chr8:127959469-127961769:.
4  chr8  128044769  128045569  chr8:128044769-128045569:.
[]


,chr,start,end,element,chr2,start2,end2,epifeature,gene,overlaps_tss
0,chr8,127735369,127736269,chr8:127735369-127736269:.,chr8,127734994,127736600,DHS,MYC,promoter
1,chr8,127794569,127795269,chr8:127794569-127795269:.,chr8,127794273,127795483,DHS,MYC,enhancer
2,chr8,127898369,127899769,chr8:127898369-127899769:.,chr8,127898381,127899783,DHS,MYC,enhancer
3,chr8,127959469,127961769,chr8:127959469-127961769:.,chr8,127959660,127960828,DHS,MYC,enhancer
4,chr8,128044769,128045569,chr8:128044769-128045569:.,chr8,128044844,128045379,DHS,MYC,enhancer
0,chr8,127735369,127736269,chr8:127735369-127736269:.,chr8,127734998,127736587,ATAC,MYC,promoter
1,chr8,127794569,127795269,chr8:127794569-127795269:.,chr8,127794051,127795837,ATAC,MYC,enhancer
2,chr8,127898369,127899769,chr8:127898369-127899769:.,chr8,127898348,127899861,ATAC,MYC,enhancer
3,chr8,127959469,127961769,chr8:127959469-127961769:.,chr8,127959553,127960915,ATAC,MYC,enhancer
4,chr8,127959469,127961769,chr8:127959469-127961769:.,chr8,127960991,127961833,ATAC,MYC,enhancer


Number of all sig elements 76
Number of sig elements that dont overlap epifeature 10


,chr,start,end,element,gene
0,chr11,33944868,33945268,chr11:33944868-33945268:.,CAT
1,chr11,34437468,34440568,chr11:34437468-34440568:.,CAT
0,chr11,33869368,33870768,chr11:33869368-33870768:.,LMO2
1,chr11,33880468,33883368,chr11:33880468-33883368:.,LMO2
2,chr11,33937168,33938068,chr11:33937168-33938068:.,LMO2
...,...,...,...,...,...
0,chr8,127735369,127736269,chr8:127735369-127736269:.,MYC
1,chr8,127794569,127795269,chr8:127794569-127795269:.,MYC
2,chr8,127898369,127899769,chr8:127898369-127899769:.,MYC
3,chr8,127959469,127961769,chr8:127959469-127961769:.,MYC


,chr,start,end,element,gene
0,chr11,33944868,33945268,chr11:33944868-33945268:.,CAT
1,chr11,34437468,34440568,chr11:34437468-34440568:.,CAT
0,chr11,33869368,33870768,chr11:33869368-33870768:.,LMO2
1,chr11,33880468,33883368,chr11:33880468-33883368:.,LMO2
2,chr11,33937168,33938068,chr11:33937168-33938068:.,LMO2
...,...,...,...,...,...
0,chr8,127735369,127736269,chr8:127735369-127736269:.,MYC
1,chr8,127794569,127795269,chr8:127794569-127795269:.,MYC
2,chr8,127898369,127899769,chr8:127898369-127899769:.,MYC
3,chr8,127959469,127961769,chr8:127959469-127961769:.,MYC


In [16]:
# Sort for merging
sorted_allgene_file = 'Sorted_AllGene_Epifeature_Intersection.bed'
fopen = open(sorted_allgene_file, 'w')
subprocess.call(['sort', '-k1,1', '-k2,2n', 'AllGene_Epifeature_Intersection.bed'], stdout = fopen)
fopen.close()

# Merge
merged_file = 'merged_AllGene_Epifeature_Intersection.bed'
fopen = open(merged_file, 'w')
subprocess.call(['bedtools', 'merge', '-i', sorted_allgene_file, '-c', '4,8,10', '-o', 'collapse'], stdout = fopen)
fopen.close()


In [12]:

### Process merged file
merged_df = pd.read_csv(merged_file, sep = '\t', names = ['chr','start','end','elements','epifeatures','promoter'])

promoter_merged_elements = []
dhs_dict = {'DHS':[]}
atac_dict = {'ATAC':[]}
h3k27ac_dict = {'H3K27ac':[]}

# Iterate through each merged element
for row in merged_df.itertuples():
    epifeatures = set(row.epifeatures.split(','))
    merged_element = row.chr + '_' + str(row.start) + '_' + str(row.end)

    ### Exclude all promoter ONLY elements
    promoter_status = set(row.promoter.split(','))
    if 'enhancer' not in promoter_status:
        continue

    # Record dictionaries
    for feature in epifeatures:
        if feature == 'DHS':
            dhs_dict['DHS'].append(merged_element)
        elif feature == 'ATAC':
            atac_dict['ATAC'].append(merged_element)
        elif feature == 'H3K27ac':
            h3k27ac_dict['H3K27ac'].append(merged_element)

# Combine to final dataframe. Note that lengths will likely be uneven
dhs_df = pd.DataFrame(dhs_dict)
atac_df = pd.DataFrame(atac_dict)
h3k27ac_df = pd.DataFrame(h3k27ac_dict)

upset_df = pd.concat([dhs_df, atac_df, h3k27ac_df], ignore_index = False, axis = 1)
upset_df.to_csv('UpSet_EpifeatureIntersection_NoPromoterOnly.csv', header = True, index = False)
display(upset_df)

### Get the identity of the H3K27ac-only peaks
unique_h3k27ac = set(h3k27ac_dict['H3K27ac']) - (set(atac_dict['ATAC']) | set(dhs_dict['DHS']))
print(unique_h3k27ac)


##### Do everything again, but now keeping all promoters
promoter_merged_elements = []
dhs_dict = {'DHS':[]}
atac_dict = {'ATAC':[]}
h3k27ac_dict = {'H3K27ac':[]}

# Iterate through each merged element
for row in merged_df.itertuples():
    epifeatures = set(row.epifeatures.split(','))
    merged_element = row.chr + '_' + str(row.start) + '_' + str(row.end)

    # Record dictionaries
    for feature in epifeatures:
        if feature == 'DHS':
            dhs_dict['DHS'].append(merged_element)
        elif feature == 'ATAC':
            atac_dict['ATAC'].append(merged_element)
        elif feature == 'H3K27ac':
            h3k27ac_dict['H3K27ac'].append(merged_element)

# Combine to final dataframe. Note that lengths will likely be uneven
dhs_df = pd.DataFrame(dhs_dict)
atac_df = pd.DataFrame(atac_dict)
h3k27ac_df = pd.DataFrame(h3k27ac_dict)

upset_df = pd.concat([dhs_df, atac_df, h3k27ac_df], ignore_index = False, axis = 1)
upset_df.to_csv('UpSet_EpifeatureIntersection_AllHits.csv', header = True, index = False)
#display(upset_df)

### Get the identity of the H3K27ac-only peaks
unique_h3k27ac = set(h3k27ac_dict['H3K27ac']) - (set(atac_dict['ATAC']) | set(dhs_dict['DHS']))
print(unique_h3k27ac)

,DHS,ATAC,H3K27ac
0,chr11_33880468_33883368,chr11_33880468_33883368,chr11_33880468_33883368
1,chr11_33937168_33938068,chr11_33937168_33938068,chr11_33937168_33938068
2,chr11_33940268_33943268,chr11_33940268_33943268,chr11_33940268_33943268
3,chr11_33943668_33946568,chr11_33943668_33946568,chr11_33943668_33946568
4,chr11_61814000_61818400,chr11_61814000_61818400,chr11_33947368_33947868
5,chr11_61833100_61835900,chr11_61833100_61835900,chr11_61814000_61818400
6,chr11_61841200_61842900,chr11_61841200_61842900,chr11_61833100_61835900
7,chr11_61868900_61873100,chr11_61868900_61873100,chr11_61841200_61842900
8,chr12_112005226_112006426,chr12_111994126_111994726,chr11_61868900_61873100
9,chr4_55614441_55616941,chr12_112005226_112006426,chr12_111994126_111994726


{'chr8_128055769_128056069', 'chr8_128052169_128052369', 'chr8_128053469_128054069', 'chr8_128057569_128058469', 'chr8_128066469_128066869', 'chr11_33947368_33947868', 'chr4_55602941_55606941'}
{'chr8_128055769_128056069', 'chr8_128052169_128052369', 'chr8_128053469_128054069', 'chr8_128057569_128058469', 'chr8_128066469_128066869', 'chr11_33947368_33947868', 'chr11_61813200_61813600', 'chr4_55602941_55606941'}


In [50]:
### Determine +/- 100 kb hit rate
# Note for multiple TSSs, the +/- 100 kb is based on the most upstream and downstream starts and ends, respectively.
# Obvious caveat, it is known that the MYC enhancers are further away, for example, and it happens to have more CREs
display(full_all_element_df)

full_all_element_df[['gene_chrom','gene_start','gene_end']] = ['','','']

def tss_range(cc_list):
    all_starts = []
    all_ends = []
    for cc in cc_list:
        chrom, start, end = cc.split('_')
        all_starts.append(start)
        all_ends.append(end)
    all_starts.sort()
    all_ends.sort()
    return([chrom, all_starts[0], all_ends[-1]])

for gene in set(full_all_element_df['gene'].tolist()):
    gene_chrom, gene_start, gene_end = tss_range(hg38ensembl_promoter_dict[gene])
    # These start and end coordinates are already TSS +/- 1000, so need to add another 99K to them
    gene_start = int(gene_start) - 99000
    gene_end = int(gene_end) + 99000
    if gene_start < 0:
        gene_start = 0

    full_all_element_df.loc[full_all_element_df['gene']==gene, ['gene_chrom','gene_start','gene_end']] = gene_chrom, gene_start, gene_end
    

display(full_all_element_df)

### Determine if in +/- 100 kb range
full_all_element_df['in_100kb_range'] = np.where((full_all_element_df['start'] <= full_all_element_df['gene_end']) & (full_all_element_df['end'] > full_all_element_df['gene_start']), 'in_range','out_of_range')
display(full_all_element_df)

print(full_all_element_df['in_100kb_range'].value_counts())





,chr,start,end,element,gene,gene_tss,gene_chrom,gene_start,gene_end,in_100kb_range
0,chr11,33944868,33945268,chr11:33944868-33945268:.,CAT,"[chr11_34437934_34439934, chr11_34448281_34450...",chr11,34338934,34570714,out_of_range
1,chr11,34437468,34440568,chr11:34437468-34440568:.,CAT,"[chr11_34437934_34439934, chr11_34448281_34450...",chr11,34338934,34570714,in_range
0,chr11,33869368,33870768,chr11:33869368-33870768:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range
1,chr11,33880468,33883368,chr11:33880468-33883368:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range
2,chr11,33937168,33938068,chr11:33937168-33938068:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range
...,...,...,...,...,...,...,...,...,...,...
0,chr8,127735369,127736269,chr8:127735369-127736269:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,in_range
1,chr8,127794569,127795269,chr8:127794569-127795269:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,in_range
2,chr8,127898369,127899769,chr8:127898369-127899769:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,out_of_range
3,chr8,127959469,127961769,chr8:127959469-127961769:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,out_of_range


,chr,start,end,element,gene,gene_tss,gene_chrom,gene_start,gene_end,in_100kb_range
0,chr11,33944868,33945268,chr11:33944868-33945268:.,CAT,"[chr11_34437934_34439934, chr11_34448281_34450...",chr11,34338934,34570714,out_of_range
1,chr11,34437468,34440568,chr11:34437468-34440568:.,CAT,"[chr11_34437934_34439934, chr11_34448281_34450...",chr11,34338934,34570714,in_range
0,chr11,33869368,33870768,chr11:33869368-33870768:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range
1,chr11,33880468,33883368,chr11:33880468-33883368:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range
2,chr11,33937168,33938068,chr11:33937168-33938068:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range
...,...,...,...,...,...,...,...,...,...,...
0,chr8,127735369,127736269,chr8:127735369-127736269:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,in_range
1,chr8,127794569,127795269,chr8:127794569-127795269:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,in_range
2,chr8,127898369,127899769,chr8:127898369-127899769:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,out_of_range
3,chr8,127959469,127961769,chr8:127959469-127961769:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,out_of_range


,chr,start,end,element,gene,gene_tss,gene_chrom,gene_start,gene_end,in_100kb_range
0,chr11,33944868,33945268,chr11:33944868-33945268:.,CAT,"[chr11_34437934_34439934, chr11_34448281_34450...",chr11,34338934,34570714,out_of_range
1,chr11,34437468,34440568,chr11:34437468-34440568:.,CAT,"[chr11_34437934_34439934, chr11_34448281_34450...",chr11,34338934,34570714,in_range
0,chr11,33869368,33870768,chr11:33869368-33870768:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range
1,chr11,33880468,33883368,chr11:33880468-33883368:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range
2,chr11,33937168,33938068,chr11:33937168-33938068:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range
...,...,...,...,...,...,...,...,...,...,...
0,chr8,127735369,127736269,chr8:127735369-127736269:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,in_range
1,chr8,127794569,127795269,chr8:127794569-127795269:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,in_range
2,chr8,127898369,127899769,chr8:127898369-127899769:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,out_of_range
3,chr8,127959469,127961769,chr8:127959469-127961769:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,out_of_range


in_range        68
out_of_range     8
Name: in_100kb_range, dtype: int64


In [93]:
### ABC analysis
# ABC is in hg19
ABC_file = '/Users/davidy/misc_resources/K562_AllPredictions.AvgHiC.ABC0.015.minus150.ForABCPaperV3.txt' # grep 'K562' from the original AllPredictions file, taken from Engreitz lab website resources 220725. Prefiltered for ABC scores >= 0.015
abc_header = ['chr2','start2','end2','name','class','activity_base','TargetGene','TargetGeneTSS','TargetGeneExpression','TargetGenePromoterActivityQuantile','TargetGeneIsExpressed','distance','isSelfPromoter','hic_contact','powerlaw_contact','powerlaw_contact_reference','hic_contact_pl_scaled','hic_pseudocount','hic_contact_pl_scaled_adj','ABC.Score.Numerator','ABC.Score','powerlaw.Score.Numerator','powerlaw.Score','CellType']
abc_df = pd.read_csv(ABC_file, sep = '\t', header = None, names=abc_header)
display(abc_df)

# Convert enhancers from hg38 to hg19
# Need to reset index
try:
    full_all_element_df.reset_index(inplace=True)
except:
    pass
converter = get_lifter('hg38', 'hg19')
for i, row in full_all_element_df.iterrows():
    chrom = row['chr'].split('chr')[-1]
    hg19_start = int(converter[chrom][int(row['start'])][0][1])
    hg19_end = int(converter[chrom][int(row['end'])][0][1])
    full_all_element_df.at[i, 'hg19_start'] = hg19_start
    full_all_element_df.at[i, 'hg19_end'] = hg19_end

full_all_element_df['hg19_start'] = full_all_element_df.hg19_start.astype(int)
full_all_element_df['hg19_end'] = full_all_element_df.hg19_end.astype(int)

display(full_all_element_df)

# Write to output
full_all_element_df.to_csv('all_sabeti_sig_element_hg19.bed', sep = '\t', index=False, header=False, columns=['chr','hg19_start','hg19_end','element','gene'])

### Intersect with ABC
fopen = open('ABC_sabeti_sig_cre_intersect.bed', 'w')
subprocess.call(['bedtools','intersect','-a',ABC_file,'-b','all_sabeti_sig_element_hg19.bed', '-wa','-wb'],stdout=fopen)
fopen.close()

abc_sabeti_int_df = pd.read_csv('ABC_sabeti_sig_cre_intersect.bed', sep = '\t', header=None, names = abc_header+['chr1','start1','end1','sig_element','screen_gene'])
display(abc_sabeti_int_df)

# Filter to rows where abc target gene is same as screen gene
target_gene_abc_filter = abc_sabeti_int_df.loc[abc_sabeti_int_df['TargetGene']==abc_sabeti_int_df['screen_gene']]
display(target_gene_abc_filter)
# Just get the significant elements
sig_abc_cres = set(target_gene_abc_filter['sig_element'].tolist())
print('There are {} sig CREs involved in ABC links to the screen gene'.format(len(sig_abc_cres)))
print('Important note: This includes self-self promoters!')


,chr2,start2,end2,name,class,activity_base,TargetGene,TargetGeneTSS,TargetGeneExpression,TargetGenePromoterActivityQuantile,...,powerlaw_contact,powerlaw_contact_reference,hic_contact_pl_scaled,hic_pseudocount,hic_contact_pl_scaled_adj,ABC.Score.Numerator,ABC.Score,powerlaw.Score.Numerator,powerlaw.Score,CellType
0,chr1,713968,714664,promoter|chr1:713818-714814,promoter,11.875948,LOC100288069,714068,NaN,0.828907,...,0.125372,0.123436,0.123436,0.001216,0.124652,1.480359,0.617870,1.465920,0.466976,K562-Roadmap
1,chr1,762794,763208,promoter|chr1:762644-763358,promoter,5.705419,LINC00115,762902,NaN,0.684272,...,0.125372,0.123436,0.123436,0.001216,0.124652,0.711191,0.254990,0.704254,0.247382,K562-Roadmap
2,chr1,762794,763208,promoter|chr1:762644-763358,promoter,5.705419,LINC01128,762970,NaN,0.684272,...,0.125372,0.123436,0.123436,0.001216,0.124652,0.711191,0.254989,0.704254,0.247422,K562-Roadmap
3,chr1,767063,767263,genic|chr1:766913-767413,genic,0.687655,LINC00115,762902,NaN,0.684272,...,0.125372,0.123436,0.123436,0.001216,0.124652,0.085717,0.030733,0.084881,0.029816,K562-Roadmap
4,chr1,767063,767263,genic|chr1:766913-767413,genic,0.687655,LINC01128,762970,NaN,0.684272,...,0.125372,0.123436,0.123436,0.001216,0.124652,0.085717,0.030733,0.084881,0.029821,K562-Roadmap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61976,chrX,155110348,155111404,promoter|chrX:155110198-155111554,promoter,14.282564,VAMP7,155110942,NaN,0.868991,...,0.125372,0.123436,0.051684,0.001216,0.052900,0.755549,0.369603,1.762983,0.559067,K562-Roadmap
61977,chrX,155119111,155119311,genic|chrX:155118961-155119461,genic,0.964026,VAMP7,155110942,NaN,0.868991,...,0.080734,0.079688,0.051814,0.001216,0.053030,0.051123,0.025008,0.076821,0.024361,K562-Roadmap
61978,chrX,155123292,155123492,genic|chrX:155123142-155123642,genic,2.240869,VAMP7,155110942,NaN,0.868991,...,0.056438,0.055821,0.020869,0.001216,0.022085,0.049489,0.024209,0.125086,0.039667,K562-Roadmap
61979,chrX,155231079,155231519,genic|chrX:155230929-155231669,genic,5.055058,IL9R,155227245,NaN,0.581066,...,0.125372,0.123436,0.048879,0.001216,0.050095,0.253233,0.153108,0.623976,0.256066,K562-Roadmap


,level_0,index,chr,start,end,element,gene,gene_tss,gene_chrom,gene_start,gene_end,in_100kb_range,hg19_start,hg19_end
0,0,0,chr11,33944868,33945268,chr11:33944868-33945268:.,CAT,"[chr11_34437934_34439934, chr11_34448281_34450...",chr11,34338934,34570714,out_of_range,33966415,33966815
1,1,1,chr11,34437468,34440568,chr11:34437468-34440568:.,CAT,"[chr11_34437934_34439934, chr11_34448281_34450...",chr11,34338934,34570714,in_range,34459015,34462115
2,2,0,chr11,33869368,33870768,chr11:33869368-33870768:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range,33890914,33892314
3,3,1,chr11,33880468,33883368,chr11:33880468-33883368:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range,33902014,33904914
4,4,2,chr11,33937168,33938068,chr11:33937168-33938068:.,LMO2,"[chr11_33864151_33866151, chr11_33868536_33870...",chr11,33765151,33992076,in_range,33958715,33959615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,71,0,chr8,127735369,127736269,chr8:127735369-127736269:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,in_range,128747615,128748515
72,72,1,chr8,127794569,127795269,chr8:127794569-127795269:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,in_range,128806815,128807515
73,73,2,chr8,127898369,127899769,chr8:127898369-127899769:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,out_of_range,128910615,128912015
74,74,3,chr8,127959469,127961769,chr8:127959469-127961769:.,MYC,"[chr8_127734434_127736434, chr8_127734473_1277...",chr8,127635434,127837621,out_of_range,128971715,128974015


,chr2,start2,end2,name,class,activity_base,TargetGene,TargetGeneTSS,TargetGeneExpression,TargetGenePromoterActivityQuantile,...,ABC.Score.Numerator,ABC.Score,powerlaw.Score.Numerator,powerlaw.Score,CellType,chr1,start1,end1,sig_element,screen_gene
0,chr4,56498198,56498398,genic|chr4:56498048-56498548,genic,1.953431,NMU,56502465,NaN,0.685494,...,0.066113,0.031284,0.241124,0.043353,K562-Roadmap,chr4,56498108,56500108,chr4:55631941-55633941:.,NMU
1,chr4,56500628,56501035,genic|chr4:56500478-56501185,genic,3.281618,NMU,56502465,NaN,0.685494,...,0.113985,0.053936,0.405070,0.072830,K562-Roadmap,chr4,56500608,56507108,chr4:55634441-55640941:.,NMU
2,chr4,56501494,56501811,genic|chr4:56501344-56501961,genic,7.537767,CLOCK,56413076,NaN,0.669179,...,0.029071,0.016220,0.076330,0.018503,K562-Roadmap,chr4,56500608,56507108,chr4:55634441-55640941:.,NMU
3,chr4,56501494,56501811,genic|chr4:56501344-56501961,genic,7.537767,EXOC1,56719815,NaN,0.772405,...,0.023986,0.017692,0.034844,0.012954,K562-Roadmap,chr4,56500608,56507108,chr4:55634441-55640941:.,NMU
4,chr4,56501494,56501811,genic|chr4:56501344-56501961,genic,7.537767,LOC644145,56686236,NaN,0.512078,...,0.018539,0.017261,0.040297,0.019854,K562-Roadmap,chr4,56500608,56507108,chr4:55634441-55640941:.,NMU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,chrX,48658948,48660968,promoter|chrX:48658798-48661118,promoter,66.907834,HDAC6,48660422,NaN,0.964437,...,1.844980,0.405871,8.258839,0.579521,K562-Roadmap,chrX,48658604,48659704,chrX:48800197-48801297:.,GATA1
210,chrX,48658948,48660968,promoter|chrX:48658798-48661118,promoter,66.907834,HDAC6,48660422,NaN,0.964437,...,1.844980,0.405871,8.258839,0.579521,K562-Roadmap,chrX,48660104,48661404,chrX:48801697-48802997:.,HDAC6
211,chrX,48658948,48660968,promoter|chrX:48658798-48661118,promoter,66.907834,GLOD5,48620153,NaN,0.516152,...,0.525875,0.154774,1.358761,0.189810,K562-Roadmap,chrX,48658704,48659504,chrX:48800297-48801097:.,HDAC6
212,chrX,48658948,48660968,promoter|chrX:48658798-48661118,promoter,66.907834,GLOD5,48620153,NaN,0.516152,...,0.525875,0.154774,1.358761,0.189810,K562-Roadmap,chrX,48658604,48659704,chrX:48800197-48801297:.,GATA1


,chr2,start2,end2,name,class,activity_base,TargetGene,TargetGeneTSS,TargetGeneExpression,TargetGenePromoterActivityQuantile,...,ABC.Score.Numerator,ABC.Score,powerlaw.Score.Numerator,powerlaw.Score,CellType,chr1,start1,end1,sig_element,screen_gene
0,chr4,56498198,56498398,genic|chr4:56498048-56498548,genic,1.953431,NMU,56502465,NaN,0.685494,...,0.066113,0.031284,0.241124,0.043353,K562-Roadmap,chr4,56498108,56500108,chr4:55631941-55633941:.,NMU
1,chr4,56500628,56501035,genic|chr4:56500478-56501185,genic,3.281618,NMU,56502465,NaN,0.685494,...,0.113985,0.053936,0.405070,0.072830,K562-Roadmap,chr4,56500608,56507108,chr4:55634441-55640941:.,NMU
5,chr4,56501494,56501811,genic|chr4:56501344-56501961,genic,7.537767,NMU,56502465,NaN,0.685494,...,0.261819,0.123889,0.930432,0.167288,K562-Roadmap,chr4,56500608,56507108,chr4:55634441-55640941:.,NMU
8,chr4,56503335,56503683,intergenic|chr4:56503185-56503833,intergenic,5.990524,NMU,56502465,NaN,0.685494,...,0.208077,0.098459,0.739447,0.132949,K562-Roadmap,chr4,56500608,56507108,chr4:55634441-55640941:.,NMU
9,chr4,56504006,56504690,intergenic|chr4:56503856-56504840,intergenic,6.244052,NMU,56502465,NaN,0.685494,...,0.216883,0.102625,0.770741,0.138576,K562-Roadmap,chr4,56500608,56507108,chr4:55634441-55640941:.,NMU
29,chr6,109624905,109626007,genic|chr6:109624755-109626157,genic,32.981854,CD164,109704015,NaN,0.979469,...,0.148358,0.045731,0.370742,0.044417,K562-Roadmap,chr6,109624348,109627748,chr6:109303145-109306545:.,CD164
43,chr6,109702301,109702501,genic|chr6:109702151-109702651,genic,3.533557,CD164,109704015,NaN,0.979469,...,0.109741,0.033828,0.436168,0.052255,K562-Roadmap,chr6,109702448,109703848,chr6:109381245-109382645:.,CD164
44,chr6,109703367,109704662,promoter|chr6:109703217-109704812,promoter,32.565964,CD164,109704015,NaN,0.979469,...,1.011395,0.311763,4.019814,0.481593,K562-Roadmap,chr6,109702448,109703848,chr6:109381245-109382645:.,CD164
46,chr8,128747485,128748554,promoter|chr8:128747335-128748704,promoter,17.306193,MYC,128748314,NaN,0.854734,...,0.404621,0.124260,2.136208,0.324301,K562-Roadmap,chr8,128747615,128748515,chr8:127735369-127736269:.,MYC
53,chr8,128910706,128911890,genic|chr8:128910556-128912040,genic,49.550089,MYC,128748314,NaN,0.854734,...,0.195147,0.059930,0.295187,0.044813,K562-Roadmap,chr8,128910615,128912015,chr8:127898369-127899769:.,MYC


There are 33 sig CREs involved in ABC links to the screen gene
Important note: This includes self-self promoters!
